In [2]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv("course_lead_scoring.csv")

In [4]:
df.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score',
       'converted'],
      dtype='object')

In [5]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [15]:
# check if the missing values are presented in the features.
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [16]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [17]:
categorical = list(df.columns[df.dtypes == 'object'])
numerical = list(df.columns[df.dtypes != 'object'])

print(categorical)
print(numeric)

['lead_source', 'industry', 'employment_status', 'location']
['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score', 'converted']


In [18]:
# If there are missing values:
# For caterogiral features, replace them with 'NA'
# For numerical features, replace with with 0.0

for category in categorical:
    df[category] = df[category].fillna('NA')

for num in numerical:
    df[num] = df[num].fillna(0.0)

df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [19]:
# What is the most frequent observation (mode) for the column industry
df.industry.value_counts()

industry
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
NA               134
Name: count, dtype: int64

## Question 2
- Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.
- What are the two features that have the biggest correlation?
  - interaction_count and lead_score
  - number_of_courses_viewed and lead_score
  - number_of_courses_viewed and interaction_count
  - annual_income and interaction_count

In [22]:
# interaction_count and lead_score
# number_of_courses_viewed and lead_score
df[['interaction_count', 'number_of_courses_viewed']].corrwith(df.lead_score)

interaction_count           0.009888
number_of_courses_viewed   -0.004879
dtype: float64

In [37]:
# number_of_courses_viewed and interaction_count
# annual_income and interaction_count
df[['annual_income', 'number_of_courses_viewed']].corrwith(df.interaction_count)

annual_income               0.027036
number_of_courses_viewed   -0.023565
dtype: float64

## Split the data
- Split your data in train/val/test sets with 60%/20%/20% distribution.
- Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
- Make sure that the target value converted is not in your dataframe.

In [51]:
from sklearn.model_selection import train_test_split

df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [52]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [53]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [54]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

In [55]:
del df_train['converted']
del df_val['converted']
del df_test['converted']

In [28]:
from sklearn.metrics import mutual_info_score

In [29]:
# Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only.
# Round the scores to 2 decimals using round(score, 2).
def mutual_info_converted(series):
    return mutual_info_score(series, y_train)

mi = df_train[categorical].apply(mutual_info_converted)
mi.sort_values(ascending=False)

lead_source          0.035396
employment_status    0.012938
industry             0.011575
location             0.004464
dtype: float64

- Now let's train a logistic regression.
- Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
- Fit the model on the training dataset.
- To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
- model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
- Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

In [31]:
from sklearn.feature_extraction import DictVectorizer

In [46]:
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(train_dicts)
X_val   = dv.transform(val_dicts)

In [47]:
val_dicts   = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
X_val

array([[5.2220e+04, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        3.0000e+00],
       [5.9656e+04, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        3.0000e+00],
       [5.7134e+04, 0.0000e+00, 0.0000e+00, ..., 1.0000e+00, 0.0000e+00,
        0.0000e+00],
       ...,
       [7.4166e+04, 0.0000e+00, 1.0000e+00, ..., 0.0000e+00, 1.0000e+00,
        1.0000e+00],
       [3.9103e+04, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        2.0000e+00],
       [4.7129e+04, 1.0000e+00, 0.0000e+00, ..., 0.0000e+00, 1.0000e+00,
        1.0000e+00]], shape=(293, 31))

In [42]:
from sklearn.linear_model import LogisticRegression

In [63]:
# Fit the model on the training dataset.
# To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
# model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
# Calculate the accuracy on the validation dataset and round it to 2 decimal digits.
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)
y_pred_proba = model.predict_proba(X_val)[:, 1]
decision = (y_pred_proba >= 0.5).astype(int)
(y_val == decision).mean(), round((y_val == decision).mean(), 3)

(np.float64(0.6996587030716723), np.float64(0.7))

## Question 5
- Let's find the least useful feature using the feature elimination technique.
- Train a model using the same features and parameters as in Q4 (without rounding).
- Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
- For each feature, calculate the difference between the original accuracy and the accuracy without the feature.


- Which of following feature has the smallest difference

In [60]:
categorical = list(df.columns[df.dtypes == 'object'])
numerical = [c for c in df_train.columns if c not in categorical]

In [66]:
original_accuracy =  (y_val == decision).mean()
targets = ['industry','employment_status','lead_score']

for feat in targets:
    feats = [c for c in (categorical + numerical) if c != feat]
    val_dicts   = df_val[feats].to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(train_dicts)
    X_val   = dv.transform(val_dicts)
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    decision = (y_pred_proba >= 0.5).astype(int)
    accuracy = (y_val == decision).mean()
    print(f"feat {feat} - Accuracy: {accuracy}, Diff: {original_accuracy - accuracy}")

feat industry - Accuracy: 0.6996587030716723, Diff: 0.0
feat employment_status - Accuracy: 0.6962457337883959, Diff: 0.0034129692832763903
feat lead_score - Accuracy: 0.7064846416382252, Diff: -0.0068259385665528916


- Now let's train a regularized logistic regression.
- Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
- Train models using all the features as in Q4.
- Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

In [69]:
categorical = list(df.columns[df.dtypes == 'object'])
numerical = [c for c in df_train.columns if c not in categorical]

for c in [0.01, 0.1, 1, 10, 100]:
    train_dicts = df_train[categorical + numerical].to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(train_dicts)
    val_dicts   = df_val[categorical + numerical].to_dict(orient='records')
    X_val = dv.transform(val_dicts)
    model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    decision = (y_pred_proba >= 0.5).astype(int)
    print(f"param {c} : {round((y_val == decision).mean(), 4)}")

param 0.01 : 0.6997
param 0.1 : 0.6997
param 1 : 0.6997
param 10 : 0.6997
param 100 : 0.6997
